<a href="https://colab.research.google.com/github/KarthikNA/Colorisation-Style-Transfer-of-Images/blob/master/video_application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# //image loader//
import cv2
vidcap = cv2.VideoCapture('/content/drive/My Drive/BigDataProject/lion.mp4')
success,image = vidcap.read()
count = 1
while success:
  cv2.imwrite("/content/drive/My Drive/BigDataProject/frames/image_frame%d.jpg" % count, image)     # save frame as JPEG file      
  success, image = vidcap.read()
  count+=1
  # print(count)

# """
# """

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25


In [ ]:
# //image loader//
# import cv2
# vidcap = cv2.VideoCapture('/content/drive/My Drive/BigDataProject/not snow - fox probably - Imgur.gif')
# success,content_image = vidcap.read()
# count = 0
# while success:
#   input_img = content_image.clone()
#   cnn = models.vgg19(pretrained=True).features.to(device).eval()
#   content_layers_default = ['conv_4']
#   style_layers_default = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']
#   cnn_normalization_mean = torch.tensor([0.485, 0.456, 0.406]).to(device)
#   cnn_normalization_std = torch.tensor([0.229, 0.224, 0.225]).to(device)

#   image = run_style_transfer(cnn, normalization_mean, normalization_std,
#                        content_img, style_img, input_img, num_steps=300,
#                        style_weight=1000000, content_weight=1)
#   cv2.imwrite("/content/drive/My Drive/BigDataProject/fox_frames/fox%d.jpg" % count, image)     # save frame as JPEG file      
#   success, content_image = vidcap.read()
#   count += 1

# print(count)

# """
# """

In [ ]:
# importing libraries to implement style-transfer

from __future__ import print_function

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from PIL import Image
import matplotlib.pyplot as plt

import torchvision.transforms as transforms
import torchvision.models as models

import copy
class ContentLoss(nn.Module):

    def __init__(self, target,):
        super(ContentLoss, self).__init__()
        # we 'detach' the target content from the tree used
        # to dynamically compute the gradient: this is a stated value,
        # not a variable. Otherwise the forward method of the criterion
        # will throw an error.
        self.target = target.detach()

    def forward(self, input):
        self.loss = F.mse_loss(input, self.target)
        return input

In [ ]:
def gram_matrix(input):
    a, b, c, d = input.size()  # a=batch size(=1)
    # b=number of feature maps
    # (c,d)=dimensions of a f. map (N=c*d)

    features = input.view(a * b, c * d)  # resise F_XL into \hat F_XL

    G = torch.mm(features, features.t())  # compute the gram product

    # we 'normalize' the values of the gram matrix
    # by dividing by the number of element in each feature maps.
    return G.div(a * b * c * d)

In [ ]:
# create a module to normalize input image so we can easily put it in a
# nn.Sequential
class Normalization(nn.Module):
    def __init__(self, mean, std):
        super(Normalization, self).__init__()
        # .view the mean and std to make them [C x 1 x 1] so that they can
        # directly work with image Tensor of shape [B x C x H x W].
        # B is batch size. C is number of channels. H is height and W is width.
        self.mean = torch.tensor(mean).view(-1, 1, 1)
        self.std = torch.tensor(std).view(-1, 1, 1)

    def forward(self, img):
        # normalize img
        return (img - self.mean) / self.std

In [ ]:
class StyleLoss(nn.Module):

    def __init__(self, target_feature):
        super(StyleLoss, self).__init__()
        self.target = gram_matrix(target_feature).detach()

    def forward(self, input):
        G = gram_matrix(input)
        self.loss = F.mse_loss(G, self.target)
        return input

In [ ]:
# desired depth layers to compute style/content losses :
content_layers_default = ['conv_4']
style_layers_default = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']
def get_style_model_and_losses(cnn, normalization_mean, normalization_std,
                               style_img, content_img,
                               content_layers=content_layers_default,
                               style_layers=style_layers_default):
    cnn = copy.deepcopy(cnn)

    # normalization module
    normalization = Normalization(normalization_mean, normalization_std).to(device)

    # just in order to have an iterable access to or list of content/syle
    # losses
    content_losses = []
    style_losses = []

    # assuming that cnn is a nn.Sequential, so we make a new nn.Sequential
    # to put in modules that are supposed to be activated sequentially
    model = nn.Sequential(normalization)

    i = 0  # increment every time we see a conv
    for layer in cnn.children():
        if isinstance(layer, nn.Conv2d):
            i += 1
            name = 'conv_{}'.format(i)
        elif isinstance(layer, nn.ReLU):
            name = 'relu_{}'.format(i)
            # The in-place version doesn't play very nicely with the ContentLoss
            # and StyleLoss we insert below. So we replace with out-of-place
            # ones here.
            layer = nn.ReLU(inplace=False)
        elif isinstance(layer, nn.MaxPool2d):
            name = 'pool_{}'.format(i)
        elif isinstance(layer, nn.BatchNorm2d):
            name = 'bn_{}'.format(i)
        else:
            raise RuntimeError('Unrecognized layer: {}'.format(layer.__class__.__name__))

        model.add_module(name, layer)

        if name in content_layers:
            # add content loss:
            target = model(content_img).detach()
            content_loss = ContentLoss(target)
            model.add_module("content_loss_{}".format(i), content_loss)
            content_losses.append(content_loss)

        if name in style_layers:
            # add style loss:
            target_feature = model(style_img).detach()
            style_loss = StyleLoss(target_feature)
            model.add_module("style_loss_{}".format(i), style_loss)
            style_losses.append(style_loss)

    # now we trim off the layers after the last content and style losses
    for i in range(len(model) - 1, -1, -1):
        if isinstance(model[i], ContentLoss) or isinstance(model[i], StyleLoss):
            break

    model = model[:(i + 1)]

    return model, style_losses, content_losses

In [ ]:
def get_input_optimizer(input_img):
    # this line to show that input is a parameter that requires a gradient
    optimizer = optim.LBFGS([input_img.requires_grad_()])
    return optimizer


def run_style_transfer(cnn, normalization_mean, normalization_std,
                       content_img, style_img, input_img, num_steps=300,
                       style_weight=1000000, content_weight=1):
    """Run the style transfer."""
    print('Building the style transfer model..')
    model, style_losses, content_losses = get_style_model_and_losses(cnn,
        normalization_mean, normalization_std, style_img, content_img)
    optimizer = get_input_optimizer(input_img)

    print('Optimizing..')
    run = [0]
    while run[0] <= num_steps:

        def closure():
            # correct the values of updated input image
            input_img.data.clamp_(0, 1)

            optimizer.zero_grad()
            model(input_img)
            style_score = 0
            content_score = 0

            for sl in style_losses:
                style_score += sl.loss
            for cl in content_losses:
                content_score += cl.loss

            style_score *= style_weight
            content_score *= content_weight

            loss = style_score + content_score
            loss.backward()

            run[0] += 1
            if run[0] % 50 == 0:
                print("run {}:".format(run))
                print('Style Loss : {:4f} Content Loss: {:4f}'.format(
                    style_score.item(), content_score.item()))
                print()

            return style_score + content_score

        optimizer.step(closure)

    # a last correction...
    input_img.data.clamp_(0, 1)

    return input_img

In [ ]:
# import cv2
# vidcap = cv2.VideoCapture('/content/drive/My Drive/BigDataProject/not snow - fox probably - Imgur.gif')
# success,image = vidcap.read()
# count = 0
# while success:
#   cv2.imwrite("/content/drive/My Drive/BigDataProject/fox_frames/frame_%d.jpeg" % count, image)     # save frame as JPEG file      
#   success,image = vidcap.read()
#   count += 1

# print(count)

In [ ]:
from torchvision.utils import save_image 
import torch.nn.functional as nnf
# desired size of the output image
# imsize = 1024 if torch.cuda.is_available() else 128  # use small size if no gpu

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
imsize = 512 
# loader = transforms.Compose([
#     transforms.Resize(imsize),  # scale imported image
#     transforms.ToTensor()])  # transform it into a torch tensor

# def image_loader(image_name):
#     image = Image.open(image_name)
#     # fake batch dimension required to fit network's input dimensions
#     image = loader(image).unsqueeze(0)
#     return image.to(device, torch.float)


loader = transforms.Compose([
    transforms.Resize([imsize, imsize]),  # scale imported image
    transforms.ToTensor()])  # transform it into a torch tensor


def image_loader(image_name):
    image = Image.open(image_name).convert('RGB')
    # fake batch dimension required to fit network's input dimensions
    image = loader(image).unsqueeze(0)
    # print(image.size())
    return image.to(device, torch.float)

style_img = image_loader("/content/drive/My Drive/BigDataProject/style_image.jpg")

count = 0
while True:
  cnn = models.vgg19(pretrained=True).features.to(device).eval()
  content_layers_default = ['conv_4']
  style_layers_default = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']
  normalization_mean = torch.tensor([0.485, 0.456, 0.406]).to(device)
  normalization_std = torch.tensor([0.229, 0.224, 0.225]).to(device)
  content_image = image_loader('/content/drive/MyDrive/BigDataProject/fox_frames/frame_%d.jpeg' % count)
  input_img = content_image.clone()
  image = run_style_transfer(cnn, normalization_mean, normalization_std,
                       content_image, style_img, input_img, num_steps=200,
                       style_weight=1000000, content_weight=1)
  
  save_image(image,"/content/drive/My Drive/BigDataProject/fox_try2/fox_style_%d.jpg" % count)     # save frame as JPEG file      
  count += 1
  print("On image %d" % count)


Building the style transfer model..
Optimizing..


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.


Streaming output truncated to the last 5000 lines.

run [200]:
Style Loss : 10.185988 Content Loss: 13.070410

On image 20
Building the style transfer model..
Optimizing..
run [50]:
Style Loss : 134.839935 Content Loss: 15.354704

run [100]:
Style Loss : 35.117970 Content Loss: 14.643974

run [150]:
Style Loss : 17.195429 Content Loss: 14.075405

run [200]:
Style Loss : 10.589800 Content Loss: 13.158594

On image 21
Building the style transfer model..
Optimizing..
run [50]:
Style Loss : 126.938148 Content Loss: 14.738462

run [100]:
Style Loss : 33.273899 Content Loss: 14.541108

run [150]:
Style Loss : 16.236366 Content Loss: 13.979025

run [200]:
Style Loss : 9.738638 Content Loss: 13.060715

On image 22
Building the style transfer model..
Optimizing..
run [50]:
Style Loss : 135.840805 Content Loss: 15.088716

run [100]:
Style Loss : 36.778648 Content Loss: 14.643009

run [150]:
Style Loss : 17.282499 Content Loss: 14.152179

run [200]:
Style Loss : 10.238301 Content Loss: 13.300577


FileNotFoundError: ignored

In [ ]:
  import cv2
  import numpy as np
  from PIL import Image

  # choose codec according to format needed
  fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
  video=cv2.VideoWriter('stylized_fox.avi', fourcc, 24,(1024,1024))

  for j in range(0,353):
     img = cv2.imread('/content/drive/My Drive/BigDataProject/fox_try2/fox_style_'+str(j)+'.jpg')
     frame = cv2.resize(img, (1024,1024))
     video.write(frame)

  cv2.destroyAllWindows()
  video.release()